# Environment Setup

In [ ]:
import importlib
import subprocess
import sys

libraries = ["langchain", "langchain_community", "huggingface_hub", "langchain_openai"]

for library in libraries:
    try:
        # Try to import the library
        module = importlib.import_module(library)
        print(f"Library {library} version: {module.__version__}")
    except ImportError:
        # If library is not installed, attempt to install it
        print(f"Library {library} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", library])
        # After installing, import again and print the version
        module = importlib.import_module(library)
        # print(f"Library {library} version after installation: {module.__version__}")
    except AttributeError:
        # If library doesn't have __version__ attribute
        print(f"Library {library} does not have a __version__ attribute.")

Library langchain version: 0.3.15
Library langchain_community version: 0.3.15
Library huggingface_hub version: 0.27.1
Library langchain_openai does not have a __version__ attribute.


## Azure Open AI

In [ ]:
import os
from langchain_openai import AzureOpenAI
from google.colab import userdata

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZ_OPENAI_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] =  "https://azopenai-demo.openai.azure.com/"

llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
embedding_model = AzureOpenAI(deployment_name="dp-text-embedding-ada-002", model_name="text-embedding-ada-002")

## Pinecone

In [ ]:
pinecone_key = userdata.get('pinecone_api_key')
pinecone_key

'fe549fc0-0058-4b43-bc21-71e37054dd9c'

In [ ]:
import langchain
import langchain_community
print("langchain.__version__", langchain.__version__)
print("langchain_community.__version__", langchain_community.__version__)

langchain.__version__ 0.3.15
langchain_community.__version__ 0.3.15


# Output Parser

## Structured output parsing

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_openai import AzureOpenAI
from langchain import PromptTemplate, LLMChain

# Define response schemas
response_schemas = [
    ResponseSchema(name="answer", description="Answer to the user's question"),
    ResponseSchema(name="fact", description="An interesting fact about the answer")
]

# Initialize the parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get formatting instructions for prompts
format_instructions = output_parser.get_format_instructions()

# Example usage with an LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Create a prompt template
prompt_template = PromptTemplate(
    template="What is the powerhouse of the cell? {format_instructions}",
    input_variables=["format_instructions"],
)
prompt = prompt_template.format(format_instructions=format_instructions)

# Create an LLMChain to integrate the parser
chain = LLMChain(llm=llm, prompt=prompt_template, output_parser=output_parser)

# Run the chain to get structured output
structured_output = chain.invoke(input=format_instructions)
print(structured_output)


{'format_instructions': 'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"answer": string  // Answer to the user\'s question\n\t"fact": string  // An interesting fact about the answer\n}\n```', 'text': {'answer': 'The powerhouse of the cell is the mitochondria.', 'fact': 'Mitochondria are responsible for converting food into energy in the form of ATP.'}}


## Custom output parsers for specific data formats

### Parsing JSON data

In [ ]:
from langchain.schema import BaseOutputParser
import json

class JSONOutputParser(BaseOutputParser):
    def parse(self, text: str):
        try:
            return json.loads(text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}")

    def get_format_instructions(self) -> str:
        return "Provide the output in a valid JSON format."

# Usage
parser = JSONOutputParser()

model_output = '{"name": "Alice", "age": 30}'  # Example raw output
parsed_data = parser.parse(model_output)
print(parsed_data)

{'name': 'Alice', 'age': 30}


### Handling SQL Queries

In [ ]:
from langchain.schema import BaseOutputParser
class SQLParser(BaseOutputParser):
    def parse(self, text: str):
        if not text.lower().startswith("select"):
            raise ValueError("Invalid SQL query. Expected a SELECT statement.")
        return text.strip()

    def get_format_instructions(self) -> str:
        return "Generate a valid SQL SELECT query."

# Usage
parser = SQLParser()

model_output = "SELECT * FROM users WHERE age > 25;"  # Example raw output
parsed_sql = parser.parse(model_output)
print(parsed_sql)


SELECT * FROM users WHERE age > 25;


## Error Handling in Output Parsing

### Structured Output Validation

In [ ]:
from langchain.schema import BaseOutputParser
from langchain.schema.output_parser import OutputParserException

class MyCustomParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        if not isinstance(text, str):
            raise OutputParserException("Expected string input")

        if not text.startswith("Result:"):
            raise OutputParserException("Output must start with 'Result:'")

        return text.split("Result:", 1)[1].strip()

# Example usage
parser = MyCustomParser()
try:
    parsed_output = parser.parse("Result: This is valid output")
    print(f"Success: {parsed_output}")
except OutputParserException as e:
    print(f"Error: {e}")


Success: This is valid output


## Integrating Custom Parsers with LangChain

### With LCEL

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI  # Or your preferred LLM
from langchain.schema.runnable import RunnablePassthrough

# Assuming JSONOutputParser is defined as in your previous code

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    "Extract data in the following format: {format_instructions}\n{input}"
)

# Initialize the model
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Build the LCEL chain
chain = (
    RunnablePassthrough()  # Pass the input through
    | prompt_template  # Apply the prompt template
    | llm  # Invoke the LLM
    | JSONOutputParser()  # Parse the output using your custom parser
)

# Example usage:
input_data = "Extract the name and age of the person from this text: Sachin is 51 years old."
output = chain.invoke(
    {"input": input_data, "format_instructions": '{"name": "string", "age": "integer"}'}
)
print(output)  # Parsed JSON data

{'name': 'Sachin', 'age': 51}


### Without LCEL

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
import json
from langchain.chat_models import ChatOpenAI  # Example LLM

# Define the JSONOutputParser
class JSONOutputParser(BaseOutputParser):
    def parse(self, text: str):
        try:
            return json.loads(text)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}")

    def get_format_instructions(self) -> str:
        return "Provide the output in a valid JSON format."

# Define the LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Define the prompt and chain
prompt = PromptTemplate(template="Extract data in the following format: {format_instructions}\n{input}")
chain = LLMChain(prompt=prompt, llm=llm, output_parser=JSONOutputParser())

# Input
input_data = "Extract the name and age of the person from this text: Sachin is 51 years old."

# Run the chain
output = chain.run(input=input_data, format_instructions='{"name": "string", "age": "integer"}')
print(output)  # Parsed JSON data


{'name': 'Sachin', 'age': 51}


# Memory Components

## Memory in chains

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.schema.runnable import RunnablePassthrough

# Initialize the OpenAI LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Initialize memory
memory = ConversationBufferMemory()

# Create a conversation chain with memory - use ConversationChain instead
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True  # Optional: set to True to see the chain's internal workings
)

# Interact with the chain
print(conversation_chain.predict(input="Hi! What is LangChain?"))
print(conversation_chain.predict(input="Can you explain its memory components?"))
print(conversation_chain.predict(input="What did I just ask you?"))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi! What is LangChain?
AI:

> Finished chain.
 Hello! LangChain is a blockchain platform that focuses on language learning and education. It was created by a team of linguists and blockchain experts with the goal of making language learning more accessible, interactive, and personalized. The platform uses a combination of AI technology and user-generated content to provide a comprehensive learning experience for users. It also has a built-in reward system that incentivizes users to actively participate and contribute to the community.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human an

## Memory in agents

In [ ]:
# Memory in Agents
from langchain.memory import ConversationSummaryMemory
from langchain.agents import initialize_agent, Tool
from langchain_openai import AzureOpenAI

# Initialize memory
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
memory = ConversationSummaryMemory(llm=llm)

# Define tools for the agent
tools = [
    Tool(
        name="Search",
        func=lambda query: f"Searching for {query}...",
        description="Simulates a search engine."
    )
]

# Create an agent with memory

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    memory=memory,
    verbose=True
)

# Interact with the agent
print(agent.run("What is LangChain?"))
print(agent.run("Can you help me search for examples of memory in LangChain?"))
print(agent.run("What did I ask you earlier?"))  # Refers back to memory




> Entering new AgentExecutor chain...
 I should use search to look up information on LangChain
Action: Search
Action Input: LangChain
Observation: Searching for LangChain...
Thought: I should read through the top results to gather information
Action: Search
Action Input: LangChain
Observation: Searching for LangChain...
Thought: I should click on the first or second result to get the most reliable information
Action: Search
Action Input: LangChain
Observation: Searching for LangChain...
Thought: I should read through the information on the website to understand what LangChain is
Action: Search
Action Input: LangChain
Observation: Searching for LangChain...
Thought: After reading through multiple sources, I now have a better understanding of LangChain and its purpose
Final Answer: LangChain is a blockchain-based platform that aims to connect language learners with native speakers for language exchange and learning.

> Finished chain.
LangChain is a blockchain-based platform that aims 

## Managing Long-Term Memory and Context

### Memory with token limit and language model

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.chains import ConversationChain
from langchain.llms import OpenAI

# Initialize memory with token limit and language model
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
memory = ConversationTokenBufferMemory(
    llm=llm,  # Add the language model here
    max_token_limit=100
)

# Create the conversation chain
conversation = ConversationChain(
    llm=llm,
    memory=memory)

# Interact with the chain
print(conversation.run("List memory components in LangChain."))
print("\n\n")
print(conversation.run("What are the benefits of token buffer memory?"))

 LangChain is a programming language that is designed for blockchain applications. It is primarily used for smart contracts and decentralized applications. As such, its memory components are optimized for these types of applications. The main memory component in LangChain is the blockchain itself, which stores all the data and transactions in a decentralized and secure manner. In addition, LangChain also utilizes a virtual machine for executing smart contracts, which also contains a memory component for storing and retrieving data. Lastly, LangChain has a built-in storage system for storing larger amounts of data, such as files or images, within the blockchain.



 Token buffer memory, also known as token bucket memory, is a type of memory used in computer networking and telecommunications. It is used to manage network traffic and ensure that data is transmitted smoothly and efficiently. The main benefit of token buffer memory is that it helps to regulate the flow of data on a network,

### Memory with summarization

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

# Initialize memory with summarization
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
memory = ConversationSummaryMemory(llm=llm)
conversation = ConversationChain(
    llm=llm,
    memory=memory
)

# Interact with the chain
print(conversation.run("Tell me about LangChain."))
print(conversation.run("What did we discuss earlier?"))  # Refers to summary


 LangChain is a decentralized platform for language learning. It utilizes blockchain technology to create a secure and transparent environment for users to learn and practice languages. The platform offers a variety of features, including personalized learning plans, virtual classrooms, and language exchange opportunities. Users can also earn rewards and certifications through their progress on the platform. LangChain has partnerships with top language schools and organizations to ensure high-quality content and resources for its users.
 We were discussing LangChain, a decentralized platform for language learning that utilizes blockchain technology. We talked about its features such as personalized learning plans, virtual classrooms, and language exchange opportunities. We also mentioned that users can earn rewards and certifications on the platform and that it has partnerships with top language schools and organizations. Is there anything else you would like to know about LangChain?


### Using Token-Limited Memory:

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.schema.runnable import RunnablePassthrough

# Initialize the OpenAI LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Initialize memory
memory = ConversationBufferMemory(max_token_limit=100)

# Create a conversation chain with memory - use ConversationChain instead
conversation_chain = ConversationChain(
    llm=llm,
    memory=memory,

)

# Interact with the chain
print(conversation.run("Explain memory components in LangChain."))
print(conversation.run("What are the benefits of token buffer memory?"))


 Sure, LangChain uses advanced memory components to help users retain and recall new language information more effectively. These components include spaced repetition algorithms, which schedule review sessions based on each user's individual learning curve, and mnemonic devices, which use associations and visual aids to improve memory retention. Additionally, the platform utilizes immersive learning techniques such as virtual reality and interactive games to engage and stimulate the brain for better language retention.
 Token buffer memory is a critical component of LangChain's advanced memory system. It allows users to store and retrieve tokens related to their language learning progress. This not only helps with organization and tracking of their learning, but also provides motivation and a sense of accomplishment as they earn and accumulate more tokens. Additionally, the token buffer memory is integrated with the spaced repetition algorithms, mnemonic devices, and immersive learning

### Hybrid Approaches

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

# Initialize the OpenAI LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Initialize hybrid memory
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150
)
conversation = ConversationChain(
    llm=llm,
    memory=memory
)

# Interact with the chain
print(conversation.run("Tell me about Artificial Intelligence."))
print(conversation.run("What did we discuss earlier?"))


 Artificial Intelligence, or AI, is a branch of computer science that focuses on creating intelligent machines that can think and act like humans. It involves the development of algorithms and software programs that can learn from data, make decisions, and solve problems on their own. AI is a rapidly growing field and has many applications in various industries, such as healthcare, finance, and transportation. Some examples of AI technologies include machine learning, natural language processing, and computer vision. AI systems can be categorized as either weak AI, which is designed for specific tasks, or strong AI, which aims to replicate human-like intelligence. Are there any specific aspects of AI you would like to know more about?
 We discussed Artificial Intelligence, its definition and applications in different industries. We also talked about the two types of AI - weak and strong. Is there anything else you would like to know?


# Embeddings and Vector Stores

## Setting up an Embedding Model in LangChain

In [ ]:
# from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, AzureOpenAIEmbeddings

# # OpenAI Embedding
# embedding_model = AzureOpenAIEmbeddings(
#     azure_deployment = "dp-text-embedding-ada-002",
#     openai_api_version = "2024-05-01-preview"
# )

# # HuggingFace Embedding
# # hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Choose based on your use case
# text = "LangChain is a framework for building AI applications."
# vector = embedding_model.embed_query(text)
# vector
# print(len(vector), vector[:5])

# Creating and Managing Vector Stores

### Create a Vector Store (Local Vector Store)

In [ ]:
!pip install -qU faiss-cpu

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize Azure OpenAI Embeddings
embedding_model = AzureOpenAIEmbeddings(
    azure_deployment = "dp-text-embedding-ada-002",
    openai_api_version = "2024-05-01-preview"
)

# Create texts and vector store
texts = ["LangChain is great for AI.", "Vector databases are powerful."]
vector_store = FAISS.from_texts(texts, embedding_model)

# Save the vector store locally
vector_store.save_local("/content/faiss_store/")

# Load the vector store
loaded_vector_store = FAISS.load_local(
    "/content/faiss_store/",
    embedding_model,
    allow_dangerous_deserialization=True
)

#### Managing Vector Stores

In [ ]:
# a. Adding New Data
new_texts = ["LangChain supports agents.", "FAISS is lightweight."]
vector_store.add_texts(new_texts)

# b. Searching for Similar Data
query = "What is LangChain?"
results = vector_store.similarity_search(query, k=2)  # Retrieve top 2 results
for result in results:
    print(result.page_content)

In [ ]:
# Saving and Loading Vector Stores
# For persistence, save the vector store and reload it when needed.

vector_store.save_local("faiss_store")
vector_store = loaded_vector_store = FAISS.load_local(
    "faiss_store",
    embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
query = "What is LangChain?"
results = loaded_vector_store.similarity_search(query, k=2)  # Get top 2 similar texts

for result in results:
    print(result.page_content)


### Using Pinecone (Cloud-based Vector Store)

In [ ]:
!pip install -qU langchain-openai langchain-pinecone

In [ ]:
import os
import getpass
from pinecone import Pinecone
from langchain_openai import AzureOpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

os.environ["PINECONE_API_KEY"] = "<insert key here>"

# Create Azure OpenAI embeddings
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment="dp-text-embedding-ada-002",
    openai_api_version=os.environ["OPENAI_API_VERSION"],
)

# Create Pinecone vector store
vector_store = PineconeVectorStore.from_texts(
    texts=["LangChain simplifies AI workflows.", "Pinecone manages vector stores."],
    embedding=embeddings,
    index_name="langchain-demo"
)

# Perform similarity search
query = "What is LangChain?"
results = vector_store.similarity_search(query, k=1)

# Print results
for result in results:
    print(result.page_content)


#### Semantic Search and metadata filtering


In [ ]:
# Filtering Results
texts_with_metadata = [
    {"text": "LangChain simplifies AI workflows.", "category": "AI"},
    {"text": "Semantic search improves search relevance.", "category": "Search"},
]

# Adding metadata to the vector store
vector_store_with_metadata = FAISS.from_texts(
    [item["text"] for item in texts_with_metadata],
    embedding_model,
    metadatas=[{"category": item["category"]} for item in texts_with_metadata]
)

# Search with a filter
query = "How does search work?"
results = vector_store_with_metadata.similarity_search(
    query, k=1, filter={"category": "Search"}
)

for result in results:
    print(result.page_content)
# example results
# Semantic search improves search relevance.

# Introduction to agents in LangChain

## Types of agents (e.g., zero-shot, conversational)


### zero-shot

In [ ]:

from langchain.tools import tool
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.llms import OpenAI
from langchain_openai import AzureOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import Tool, AgentExecutor

# Define the calculator tool
@tool
def calculator_tool(expression: str) -> str:
    """A simple calculator for evaluating expressions."""
    try:
        result = eval(expression)
        return str(result)
    except Exception:
        return "Invalid input."

# Define the tools
tools = [
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="Use this tool to evaluate mathematical expressions.",
    )
]

# Define the Azure OpenAI LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["input", "query"],
    template="""Answer the following question: {input}
{query}""",
)

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Define a query
query = "What is 12 * 8 plus 100?"

# Execute the query
response = agent.run(input=query)
print(response)


### Conversational Agent

In [ ]:

from langchain.agents import create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI

# Initialize the LLM (OpenAI GPT)
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Configure Memory for Conversation History
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize the Conversational Agent
agent_chain = initialize_agent(
    tools=[],  # Add tools here if needed (e.g., calculators, search tools)
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True  # Set to True for detailed logs of interactions.
)

# Interact with the Agent
print("Welcome to the Conversational Agent! Type 'exit' to quit.\n")

while True:
    # Get user input
    user_input = input("You: ")

    # Exit condition
    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    # Run the agent and get a response
    response = agent_chain.run(input=user_input)

    # Print the response from the agent
    print(f"Agent: {response}")


## Tools-Enabled Agent

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent

# Initialize the database
db = SQLDatabase.from_uri("sqlite:///Langchain.db")

# Initialize the language model
llm = llm

# Create the SQL agent
agent_executor = create_sql_agent(llm, db=db, verbose=True)

# Run a query
resp = agent_executor.run("Show me the first 5 rows of the 'Sample' table.")
print(resp)


## Custom Agents

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import StructuredTool
from typing import Optional
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage

# Define the currency conversion tool
def currency_conversion_tool(amount: float, from_currency: str, to_currency: str) -> Optional[float]:
    """Convert between USD, EUR and INR currencies"""
    rates = {
    "USD-INR": 83.0,
    "EUR-USD": 1.1,
    "INR-USD": 0.012,
    "USD-EUR": 0.9
}
    pair = f"{from_currency}-{to_currency}"
    if pair in rates:
        return amount * rates[pair]
    elif f"{to_currency}-{from_currency}" in rates:
        return amount / rates[f"{to_currency}-{from_currency}"]
    return None

# Create a LangChain tool with better description
currency_tool = StructuredTool.from_function(
    func=currency_conversion_tool,
    name="currency_converter",
    description="""Convert amounts between currencies (USD, EUR, INR).
    Args:
        amount (float): The amount to convert
        from_currency (str): Source currency code (USD, EUR, or INR)
        to_currency (str): Target currency code (USD, EUR, or INR)
    Returns:
        float: The converted amount
    """
)

# Initialize the LLM
llm = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Create system message for the agent
system_message = """You are a helpful currency conversion assistant.
When given a currency conversion request:
1. Extract the amount, source currency, and target currency
2. Use the currency_converter tool to perform the conversion
3. Always respond with the converted amount in a clear format
4. If there's an error, explain what went wrong

Format your response as:
{amount} {from_currency} = {converted_amount} {to_currency}"""

# Create the agent with custom configuration
agent = initialize_agent(
    tools=[currency_tool],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    system_message=system_message,
    handle_parsing_errors=True
)

# Example usage with error handling
def convert_currency(query: str) -> str:
    try:
        result = agent.run(query)
        return result if result else "Sorry, couldn't perform the conversion. Please check the currency codes."
    except Exception as e:
        return f"Error performing conversion: {str(e)}"

# Test the conversion
query = "Convert 100 EUR to USD"
print(convert_currency(query))

## Custom Prompts

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template=(
        "You are a helpful assistant specializing in customer support.\n"
        "Conversation History:\n{history}\n"
        "User Query: {input}\n"
        "Provide a detailed and polite response."
    ),
)
agent = create_structured_chat_agent(llm=llm, tools=tools, prompt=custom_prompt, memory=memory)

# Chat Models and LLMs

## Configuring and fine-tuning chat models

### Incorporating Memory for Multi-Turn Conversations

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

conversation.predict(input="Hello, can you help me plan my day?")
conversation.predict(input="Where did I go yesterday?")

' According to your GPS data, you went to work in the morning, then to the gym in the evening, and finally to a restaurant for dinner. Is there anything else you would like to know about your activities yesterday?'

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='Asia')
print(messages)  # Outputs the formatted chat messages

output = llm.invoke(messages)
print(output)

[SystemMessage(content='You respond only in the JSON format.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Top 5 countries in Asia by population.', additional_kwargs={}, response_metadata={})]

AI System: {"Countries": [{"Country": "China", "Population": "1,439,323,776"}, {"Country": "India", "Population": "1,380,004,385"}, {"Country": "Indonesia", "Population": "273,523,615"}, {"Country": "Pakistan", "Population": "220,892,340"}, {"Country": "Bangladesh", "Population": "164,689,383"}]}


# LangChain Expression Language (LCEL)

## Example 1: Basic LCEL Syntax

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
chain = prompt | model
chain.invoke({"topic": "bears"})


'\n\nAI: Why did the bear go to the doctor?\n\nBecause he was feeling grizzly!'

## Example 2: LCEL allows for the creation of more sophisticated chains.

In [ ]:
# Code Example :LCEL allows for the creation of more sophisticated chains. Here's an example that includes multiple steps:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
output_parser = StrOutputParser()
chain = prompt | model | output_parser
chain.invoke({"topic": "bears"})


'\n\nRobot: Why did the bear wear a tuxedo?\nBecause he wanted to look "bearly" dressed!'

## Example 3: Using RunnableParallel for Multiple Inputs LCEL supports parallel operations.


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain_openai import AzureOpenAI

# Define the prompt template for summarization and translation
prompt = ChatPromptTemplate.from_template(
    """Summarize the following text: {text}
    Then translate the summary to {language}."""  # Added newline for better formatting
)

# Initialize the model
model = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

# Define the RunnableParallel operation (modified)
summarize_and_translate = RunnableParallel(
    summary=prompt | model,
    # original_text=RunnablePassthrough()  # Removed as it's not used in the chain
)

# Define the complete chain (modified)
chain = summarize_and_translate  # Removed extra prompt and model invocation

# Invoke the chain with input values
result = chain.invoke({
    "text": "LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis",
    "language": "French"
})

# Print the result (optional)
print(result)

{'summary': "\n\nLangChain est un cadre logiciel qui facilite l'intégration de grands modèles de langage (LLM) dans les applications. En tant que cadre d'intégration de modèle de langage, les cas d'utilisation de LangChain chevauchent largement ceux des modèles de langage en général, y compris l'analyse et la synthèse de documents, les chatbots et l'analyse de code.\n\nLangChain est un cadre logiciel pour intégrer les grands modèles de langage dans les applications. Les cas d'utilisation de LangChain sont similaires à ceux des modèles de langage, tels que l'analyse et la synthèse de documents, les chatbots et l'analyse de code."}


## Example 4: Error Handling in LCEL

In [ ]:
from typing import Optional
from langchain.callbacks.manager import CallbackManagerForChainRun
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_openai import AzureOpenAI

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
model = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")

def fake_error_handler(error: Exception, run_manager: Optional[CallbackManagerForChainRun]) -> str:
    return "Oops! Something went wrong."

# Wrap the error handler in a RunnableLambda
runnable_error_handler = RunnableLambda(lambda x: fake_error_handler(x, None)) # type: ignore

# Modified chain with the runnable error handler
chain = prompt | model.with_fallbacks([runnable_error_handler])

# Invoke the chain to generate a joke about bears
result = chain.invoke({"topic": "bears"})
print(result) # Print or use the result as needed



AI: Why was the bear so good at math? Because he was a natural at polar-izing numbers!


## Example 5: Using LCEL with Retriever

In [ ]:
from langchain_openai import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize Azure OpenAI Embeddings
embedding_model = AzureOpenAIEmbeddings(
    azure_deployment = "dp-text-embedding-ada-002",
    openai_api_version = "2024-05-01-preview"
)

# Create texts and vector store
texts = ["LangChain is great for AI.", "Vector databases are powerful."]
vector_store = FAISS.from_texts(texts, embedding_model)

# Save the vector store locally
vector_store.save_local("/content/faiss_store/")

# Load the vector store
loaded_vector_store = FAISS.load_local(
    "/content/faiss_store/",
    embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser

## Load existing Vector DB FAISS, that we created in the section "Create a Vector Store (Local Vector Store)"
vectorstore = FAISS.load_local(
                              "/content/faiss_store/",
                              embedding_model,
                              allow_dangerous_deserialization=True
                          )

# Adding New Data
new_texts = ["harrison worked at google", "harrison likes spicy food"]
vectorstore.add_texts(new_texts)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
            {context}

            Question: {question}
            """
prompt = ChatPromptTemplate.from_template(template)

model = llm

chain = (
    RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    )
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("where did harrison work?")

'\nGoogle.'

## Example 6: LCEL advanced Chain

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

# Define the prompt template
prompt_template = ChatPromptTemplate.from_template(
    "Answer the question based on the context: {context_a} {context_b} Question: {question}"
)

# Initialize the model and output parser
model = model = AzureOpenAI(deployment_name="dp-gpt-35-turbo-instruct", model_name="gpt-35-turbo-instruct")
output_parser = StrOutputParser()

# Build the chain
chain = (
    {
        "context_a": lambda x: x["retriever_a"],
        "context_b": lambda x: x["retriever_b"],
        "question": RunnablePassthrough()
    }
    | prompt_template
    | model
    | output_parser
)

# Example inputs
inputs = {
    "question": "What are the benefits of using LangChain?",
    "retriever_a": "LangChain enables modular workflows and easy integration with APIs.",
    "retriever_b": "LangChain supports complex applications like question answering and chatbots."
}

# Execute the chain
result = chain.invoke(inputs)
print(result)



The benefits of using LangChain include enabling modular workflows and easy integration with APIs, as well as supporting complex applications like question answering and chatbots.


# End of Chapter 3